In [78]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import requests
import numpy as np

In [2]:
import platform
from matplotlib import font_manager, rc

# 코드 테스트

In [3]:
driver_path = 'C:/Users/User/Desktop/chromedriver/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(driver_path)

url = 'https://music.bugs.co.kr/chart/track/realtime/total'
driver.get(url)
time.sleep(2)

C:\Users\User\AppData\Local\Temp/ipykernel_9468/1055193298.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


## frame 추출

In [4]:
soup = bs(driver.page_source,'lxml')

In [5]:
raw_table = soup.find('table', 'list trackList byChart')
raw_table2 = raw_table.find('tbody')
len(raw_table2)

201

In [6]:
len(raw_table2.find_all('th'))

100

## 제목 리스트 만들기

In [7]:
song_name_list = []

for area in raw_table2.find_all('th'):
    name_area = area.find('p').a['title']
    song_name_text = name_area.strip()
    song_name_list.append(song_name_text)

print(len(song_name_list))
print(song_name_list[:5])

100
['봄여름가을겨울 (Still Life)', 'LOVE DIVE', 'Feel My Rhythm', 'TOMBOY', '사랑인가 봐 (Acoustic Ver.)']


## 아티스트, 앨범 리스트 만들기

In [8]:
len(raw_table2.find_all('td', 'left'))

200

In [9]:
count = 0

for area in raw_table2.find_all('td', 'left'):
    count +=1
    
    print(area.text)
    
    if count > 5: break



BIGBANG (빅뱅)



봄여름가을겨울 (Still Life)



IVE (아이브)



LOVE DIVE



Red Velvet (레드벨벳)



‘The ReVe Festival 2022 - Feel My Rhythm’



In [10]:
artist_album_list = []

for area in raw_table2.find_all('td', 'left'):
    artist_album_list.append(area.text.strip())
    
artist_album_list[:10]

['BIGBANG (빅뱅)',
 '봄여름가을겨울 (Still Life)',
 'IVE (아이브)',
 'LOVE DIVE',
 'Red Velvet (레드벨벳)',
 '‘The ReVe Festival 2022 - Feel My Rhythm’',
 '(여자)아이들',
 'I NEVER DIE',
 '김세정',
 '사랑인가 봐 (사내맞선 OST Bonus Track)']

td 태그의 class=left는 2개가 있습니다. 여기에는 가수 이름과 앨범이 포함되는데, 이들을 분리하여 각각 리스트를 만들어줍니다.

In [11]:
artist_list = artist_album_list[::2]
album_list = artist_album_list[1::2]

In [12]:
print(len(artist_list), len(album_list))

100 100


## 데이터프레임 만들기

In [13]:
rank = list(range(1, 101))

In [14]:
df = pd.DataFrame({'순위': rank,
                   '곡': song_name_list,
                   '아티스트': artist_list,
                   '앨범': album_list
                  })

df.shape

(100, 4)

In [15]:
df.head()

,순위,곡,아티스트,앨범
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,Feel My Rhythm,Red Velvet (레드벨벳),‘The ReVe Festival 2022 - Feel My Rhythm’
3,4,TOMBOY,(여자)아이들,I NEVER DIE
4,5,사랑인가 봐 (Acoustic Ver.),김세정,사랑인가 봐 (사내맞선 OST Bonus Track)


In [16]:
df.isnull().sum()

순위      0
곡       0
아티스트    0
앨범      0
dtype: int64

# 벅스뮤직 차트 크롤링 함수 만들기

In [82]:
def bugs_df(url):
    driver_path = 'C:/Users/User/Desktop/chromedriver/chromedriver_win32/chromedriver'
    driver = webdriver.Chrome(driver_path)

    driver.get(url)
    time.sleep(2)
    
    soup = bs(driver.page_source,'lxml')

    raw_table = soup.find('table', 'list trackList byChart')
    raw_table2 = raw_table.find('tbody')
    
    # 제목 리스트 만들기
    song_name_list = []

    for area in raw_table2.find_all('th'):
        name_area = area.find('p').a['title']
        song_name_text = name_area.strip()
        song_name_list.append(song_name_text)
        
    # 아티스트, 앨범 리스트 만들기
    artist_album_list = []

    for area in raw_table2.find_all('td', 'left'):
        artist_album_list.append(area.text.strip())

    artist_list = artist_album_list[::2]
    album_list = artist_album_list[1::2]
    
    # 데이터프레임 만들기
    rank = list(range(1, 101))
    
    df = pd.DataFrame({'순위': rank,
                   '곡': song_name_list,
                   '아티스트': artist_list,
                   '앨범': album_list
                  })
    
    df.head()
    
    random_number = random.uniform(1, 2)
    time.sleep(random_number)
    
    return df

## bugs_df에서 나온 df 저장하는 함수

In [87]:
from tqdm import tqdm

In [90]:
def save_df(start_date, end_date):
    # 원하는 날짜 리스트에 담기
    # pd.date_range에 원하는 날짜 입력하기
    date_period = pd.date_range(start_date, end_date)
    timestamp_list = list(date_period)

    date_list = []

    for date in timestamp_list:

        date_list.append(str(date)[:10])

    for datetime in tqdm(date_list):
        date = datetime.replace('-', '')

        url = "https://music.bugs.co.kr/chart/track/day/total?chartdate="
        url_with_date = url + date

        df = bugs_df(url_with_date)

        # DataFrame.inser(loc,column, value, allow_duplicates=False)
        df.insert(0, '날짜', datetime)
        # 파일 저장
        save_path = f"C:/Users/User/Desktop/음원사재기/벅스/day_bugs_{date}.xlsx"
        df.to_excel(save_path, index=False)

        df

In [84]:
save_df('2022-02-01', '2022-02-10')

C:\Users\User\AppData\Local\Temp/ipykernel_9468/308752351.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [104]:
df.isnull().sum()

순위      0
곡       0
아티스트    0
앨범      0
dtype: int64